In [7]:
from langchain_core.documents import Document

In [8]:
doc=Document(
    page_content='tHIS IS THE MAINTEXT CONTENT I AM USIN TO CREATE RAG',
    mrta_data={
        "source":"example.txt",
        "pages":1,
        "author":"Paras Chinchalkar",
        "date_created":"2025-01-10"

    }

)

In [9]:
#creatking a simple txt file in here
import os
os.makedirs("../data/text_files",exist_ok=True)


In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("Sample file has been created")

Sample file has been created


In [12]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader
loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [13]:
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader

dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)


In [14]:
documents=dir_loader.load()

100%|██████████| 2/2 [00:00<00:00, 1001.27it/s]

In [15]:
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n    \n    \n    '),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the m

In [16]:
pip install pymupdf

In [17]:
pip install PyMuPDFLoader

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PyMuPDFLoader (from versions: none)
ERROR: No matching distribution found for PyMuPDFLoader


In [18]:

# Install PyMuPDFLoader if not already installed

dir_loader = DirectoryLoader(
    "./data/pdf",
    glob="*.pdf",  # loads all PDFs in the folder
    loader_cls=PyMuPDFLoader,
    show_progress=False
)

pdf_documents = dir_loader.load()

In [19]:
pdf_documents

[Document(metadata={'producer': 'PD4ML 4.0.16', 'creator': '', 'creationdate': '', 'source': 'data\\pdf\\cancer_and_cure__a_critical_analysis.27.pdf', 'file_path': 'data\\pdf\\cancer_and_cure__a_critical_analysis.27.pdf', 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '.', 'keywords': 'LWW', 'moddate': "D:20231203095934Z00'00'", 'trapped': '', 'modDate': "D:20231203095934Z00'00'", 'creationDate': '', 'page': 0}, page_content='441\n© 2017 Indian Journal of Cancer | Pulished by Wolters Kluwer - Medknow\nCancer and cure: A\xa0critical analysis\nRoy PS, Saikia BJ\nDepartment of Medical Oncology, Dr.\xa0B. Borooah Cancer Institute, Guwahati, Assam, India\nCorrespondence to: Dr.\xa0Roy PS, E‑mail:\xa0drpsr.roy@rediffmail.com\nAbstract\nCancer is one of the most dreaded diseases of the 20th\xa0century and spreading further with continuance and increasing incidence in the 21st\xa0century. \nThe situation is so alarming that every fourth person is having a lifetime

In [20]:
type(pdf_documents[0])

langchain_core.documents.base.Document

In [21]:
!pip install sentence_transformers

In [22]:
pip install chormadb

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement chormadb (from versions: none)
ERROR: No matching distribution found for chormadb


In [23]:
#embedding and vectorStorDb

In [24]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
import os
import uuid
import numpy as np
from typing import List, Any
from langchain.text_splitter import TokenTextSplitter
import chromadb

class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            documents_text.append(doc.page_content)
            embeddings_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

# Initialize the vector store
vectorstore = VectorStore()


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 242


In [62]:
#vectorstore will be from here now on

class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 363


In [63]:
vectorStore

In [64]:
pip install pypdf

In [65]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


pdf_paths = [
    "./data/pdf/cancer_and_cure__a_critical_analysis.27.pdf",
    "./data/pdf/medical_oncology_handbook_june_2020_edition.pdf"
]
all_docs = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    all_docs.extend(docs)  # Combine all documents

# Step 2: Split combined documents into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(documents)

chunks = split_documents(all_docs)


In [66]:
chunks = split_documents(all_docs)

In [67]:
chunks

[Document(metadata={'producer': 'PD4ML 4.0.16', 'creator': 'PyPDF', 'creationdate': '', 'keywords': 'LWW', 'author': '', 'moddate': "D:20231203095934Z00'00'", 'subject': '.', 'source': './data/pdf/cancer_and_cure__a_critical_analysis.27.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='441© 2 017 Indian Journal of Cancer | Pulished by Wolters Kluwer - Medknow\nCancer and cure: A critical analysis\nRoy PS, Saikia BJ\nDepartment of Medical Oncology, Dr. B. Borooah Cancer Institute, Guwahati, Assam, India\nCorrespondence to: Dr. Roy PS, E‑mail: drpsr.roy@rediffmail.com\nAbstract\nCancer is one of the most dreaded diseases of the 20th century and spreading further with continuance and increasing incidence in the 21st century. \nThe situation is so alarming that every fourth person is having a lifetime risk of cancer. India registers more than 11 lakh new cases of cancer \nevery year, whereas, this figure is above 14 million worldwide. Is cancer curable? The short answer 

In [68]:
###convert the text to embddings

texts=[doc.page_content for doc in chunks]
texts

['441© 2 017 Indian Journal of Cancer | Pulished by Wolters Kluwer - Medknow\nCancer and cure: A critical analysis\nRoy PS, Saikia BJ\nDepartment of Medical Oncology, Dr. B. Borooah Cancer Institute, Guwahati, Assam, India\nCorrespondence to: Dr. Roy PS, E‑mail: drpsr.roy@rediffmail.com\nAbstract\nCancer is one of the most dreaded diseases of the 20th century and spreading further with continuance and increasing incidence in the 21st century. \nThe situation is so alarming that every fourth person is having a lifetime risk of cancer. India registers more than 11 lakh new cases of cancer \nevery year, whereas, this figure is above 14 million worldwide. Is cancer curable? The short answer to this question is “Yes.” In fact, all cancers \nare curable if they are caught early enough. Cancer cells continue to grow unless one of four things occur: (1) The cancerous mass is removed',
 'are curable if they are caught early enough. Cancer cells continue to grow unless one of four things occur: 

In [69]:
#generate the embeddings
# embeddings=embedding_manager.generate_embeddings(texts)


In [70]:
#store the vector database
# vectorStore.add_documents(chunks,embeddings)


In [83]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [84]:

### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 121 texts...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generated embeddings with shape: (121, 384)
Adding 121 documents to vector store...
Successfully added 121 documents to vector store
Total documents in collection: 605


In [85]:

class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []
        
rag_retriever=RAGRetriever(VectorStore,embedding_manager)

In [86]:
rag_retriever

In [87]:
rag_retriever.retrieve("Unified multi-task leraning frameworks?")

Retrieving documents for query: 'Unified multi-task leraning frameworks?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Error during retrieval: type object 'VectorStore' has no attribute 'collection'


[]

In [88]:
#Integration vectorDB context pipeline with LLM output


from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

True

In [89]:
groq_api_key="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"
llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",max_tokens=1024)

In [90]:
def rag_simple(query, retriever, llm, top_k=3):
    # Retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    
    if not context.strip():
        return "No relevant context found to answer the question."
    
    # Generate the answer using the LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question:
{query}

Answer:"""
    
    response = llm.invoke(prompt)
    return response.content

In [91]:
answer=rag_simple("What is attention mechanism",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is attention mechanism'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Error during retrieval: type object 'VectorStore' has no attribute 'collection'
No relevant context found to answer the question.


In [92]:
def rag_simple(query, retriever, llm, top_k=3):
    # Retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    
    if not context.strip():
        return "No relevant context found to answer the question."
    
    # Generate the answer using the LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question:
{query}

Answer:"""
    
    response = llm.invoke(prompt)
    return response.content

In [93]:
answer=rag_simple("WHat is attention mechanism",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'WHat is attention mechanism'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Error during retrieval: type object 'VectorStore' has no attribute 'collection'
No relevant context found to answer the question.
